**Görev - 1**

In [1]:
# 1. Selenium'un Webdriver sınıfını kullanarak bir adet options adında ChromeOptions tanımlamalıyız

In [2]:
from selenium import webdriver
options = webdriver.ChromeOptions()

In [3]:
# 2. Tanımladığımız options'a tam ekran özelliği kazandırıyoruz

In [4]:
options.add_argument('--start-maximized')

In [5]:
# 3. Bir önceki adımlarda hazırladığımız options'ı da kullanarak bir adet driver adında Chrome tarayıcısı oluşturuyoruz.

In [6]:
driver = webdriver.Chrome(options)

******

**Görev - 2**

In [7]:
# 1. Anasayfayı driver ile açacağız ve inceleyeceğiz.

In [8]:
import time
SLEEP_TIME = 2

In [9]:
driver.get("https://books.toscrape.com/")
time.sleep(SLEEP_TIME)

In [10]:
# 2. 'Travel' ve 'Nonfiction' kategori sayfalarının linklerini tutan elemanları bulan XPATH sorgusu yazmalıyız

In [11]:
category_elements_xpath ="//a[contains(text() , 'Travel') or contains(text() , 'Nonfiction')]"

In [12]:
# 3. XPATH sorgusu ile yakaladığımız elementleri driver'ı kullanarak bulacağız ve kategori detay linklerini kazıyacağız

In [13]:
from selenium.webdriver.common.by import By
category_elements = driver.find_elements(By.XPATH , category_elements_xpath) # find.elements metodunu kullandık

In [14]:
category_elements

[<selenium.webdriver.remote.webelement.WebElement (session="ba906400091fd3cc3c99fea7a8efa088", element="f.0F99AA6396F07A06249017B042671FE1.d.CB58A4E4D82D744954147DA7FE2C22C3.e.52")>,
 <selenium.webdriver.remote.webelement.WebElement (session="ba906400091fd3cc3c99fea7a8efa088", element="f.0F99AA6396F07A06249017B042671FE1.d.CB58A4E4D82D744954147DA7FE2C22C3.e.53")>]

In [15]:
category_urls = [element.get_attribute("href") for element in category_elements]
category_urls

['https://books.toscrape.com/catalogue/category/books/travel_2/index.html',
 'https://books.toscrape.com/catalogue/category/books/nonfiction_13/index.html']

************

**Görev - 3**

In [16]:
# 1. Herhangi bir detay sayfasına girip o sayfadaki detay linkini tutan elementleri yakalayan XPATH sorgusunu yazacağız

In [17]:
driver.get(category_urls[0])
time.sleep(SLEEP_TIME)
book_elements_xpath = "//div[@class='image_container']//a"

In [18]:
# 2. driver ile XPATH sorgusunu kullanarak elementleri yakalayacağız ve detay linklerini çıkartacağız

In [19]:
book_elements = driver.find_elements(By.XPATH , book_elements_xpath)
book_urls = [element.get_attribute("href") for element in book_elements]
book_urls
len(book_urls)

11

In [20]:
# 3. Sayfalandırma (Pagination) için butonlara tıklamak yerine sayfa linkini manipüle edeceğiz

In [21]:
MAX_PAGINATION = 3 #test amaçlı
url = category_urls[0]
book_urls = []

for i in range(1 , MAX_PAGINATION):
    update_url = url if i == 1 else url.replace("index" , f"page-{i}")
    driver.get(update_url)
    book_elements = driver.find_elements(By.XPATH, book_elements_xpath)
    
    temp_urls = [element.get_attribute("href") for element in book_elements]
    book_urls.extend(temp_urls)
    
book_urls
len(book_urls)

22

In [22]:
# 4. Sayfalandırmanın sonuna geldiğini anlamak adına kateegorinin 999. sayfasına giderek karşınıza çıkan sayfayı inceleyeceğiz

In [23]:
# 5. Bir önceki adımda incelemenin sonucunda sayfalandırmada sonsuz döngüye girmemek adına kontrol kullanmalıyız

In [24]:
MAX_PAGINATION = 4 #test amaçlı
url = category_urls[0]
book_urls = []

for i in range(1 , MAX_PAGINATION):
    update_url = url if i == 1 else url.replace("index" , f"page-{i}")
    driver.get(update_url)
    book_elements = driver.find_elements(By.XPATH, book_elements_xpath)
    
    if not book_elements:
        break
    
    temp_urls = [element.get_attribute("href") for element in book_elements]
    book_urls.extend(temp_urls)
    
book_urls
len(book_urls)

33

*************

**Görev - 4**

In [25]:
# 1. Herhangi bir ürünün detay sayfasına girip class attribute'ı content olan div elementini yakalayınız.

In [32]:
driver.get(book_urls[0])
time.sleep(SLEEP_TIME)
content_div = driver.find_elements(By.XPATH , "//div[@class='content']")

In [26]:
# 2. Yakaladığınız divin html'ini alınız ve inner_html adlı değişkene atayınız

In [38]:
inner_html = content_div[0].get_attribute("innerHTML")

In [39]:
# 3. inner_html ile soup objesi oluşturunuz

In [40]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(inner_html , "html.parser")

In [41]:
# 4. Oluşturduğunuz soup objesi ile su bilgileri kazıyınız.

In [43]:
# -> Kitap Adı,

name_elem = soup.find("h1")
book_name = name_elem.text

In [46]:
# -> Kitap Fiyatı,

price_elem = soup.find("p" , attrs={"class":"price_color"})
book_price = price_elem.text

In [48]:
# -> Kitap Yıldız Sayısı,

import re
regex = re.compile('^star-rating')
star_elem = soup.find("p" , attrs={"class":regex})
book_star_count = star_elem["class"][-1]

In [49]:
# -> Kitap Açıklaması

desc_elem = soup.find("div" , attrs={"id":"product_description"}).find_next_sibling() # bir sonraki kardeşini bul

In [51]:
# -> Product Information Başlığı Altında Kalan Tablodaki Bilgiler

product_info = {}
table_rows = soup.find("table").find_all("tr")

for row in table_rows:
    key = row.find("th").text
    value = row.find("td").text
    product_info[key] = value

In [55]:
print("name:" , book_name)
print("price:" , book_price)
print("star:" , book_star_count)
print("description:" , desc_elem)
print("info:" , product_info)

name: It's Only the Himalayas
price: £45.17
star: Two
description: <p>“Wherever you go, whatever you do, just . . . don’t do anything stupid.” —My MotherDuring her yearlong adventure backpacking from South Africa to Singapore, S. Bedford definitely did a few things her mother might classify as "stupid." She swam with great white sharks in South Africa, ran from lions in Zimbabwe, climbed a Himalayan mountain without training in Nepal, and wa “Wherever you go, whatever you do, just . . . don’t do anything stupid.” —My MotherDuring her yearlong adventure backpacking from South Africa to Singapore, S. Bedford definitely did a few things her mother might classify as "stupid." She swam with great white sharks in South Africa, ran from lions in Zimbabwe, climbed a Himalayan mountain without training in Nepal, and watched as her friend was attacked by a monkey in Indonesia.But interspersed in those slightly more crazy moments, Sue Bedfored and her friend "Sara the Stoic" experienced the sight

*****

**Görev - 5**

In [58]:
# 1. İşlemleri fonkssiyonlarştıracağız.

In [30]:
def get_book_detail(driver, url):
    driver.get(url)
    time.sleep(SLEEP_TIME)
    content_div = driver.find_elements(By.XPATH , "//div[@class='content']")

    inner_html = content_div[0].get_attribute("innerHTML")
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(inner_html , "html.parser")
    name_elem = soup.find("h1")
    book_name = name_elem.text
    price_elem = soup.find("p" , attrs={"class":"price_color"})
    book_price = price_elem.text
    import re
    regex = re.compile('^star-rating')
    star_elem = soup.find("p" , attrs={"class":regex})
    book_star_count = star_elem["class"][-1]
    desc_elem = soup.find("div" , attrs={"id":"product_description"}).find_next_sibling() # bir sonraki kardeşini bul
    book_desc = desc_elem.text
    product_info = {}
    table_rows = soup.find("table").find_all("tr")
    for row in table_rows:
        key = row.find("th").text
        value = row.find("td").text
        product_info[key] = value    
    return {'book_name' : book_name , 'book_price' : book_price , 'book_star_count' : book_star_count , 'book_desc' : book_desc,
            **product_info}   

def get_book_urls(driver, url):
    MAX_PAGINATION = 3 #test amaçlı
    book_urls = []
    book_elements_xpath = "//div[@class='image_container']//a"

    for i in range(1 , MAX_PAGINATION):
        update_url = url if i == 1 else url.replace("index" , f"page-{i}")
        driver.get(update_url)
        time.sleep(SLEEP_TIME)
        book_elements = driver.find_elements(By.XPATH, book_elements_xpath)
        
        if not book_elements:
            break
            
        temp_urls = [element.get_attribute("href") for element in book_elements]
        book_urls.extend(temp_urls)
def get_travel_and_nonfiction_category_urls(driver , url):
    driver.get(url)
    time.sleep(SLEEP_TIME)
    category_elements_xpath ="//a[contains(text() , 'Travel') or contains(text() , 'Nonfiction')]"
    category_elements = driver.find_elements(By.XPATH , category_elements_xpath) # find.elements metodunu kullandık
    category_urls = [element.get_attribute("href") for element in category_elements]
    return category_urls


def initialize_driver():
#    Initilizes driver with maximized option
    options = webdriver.ChromeOptions()
    options.add_argument('--start-maximized');
    driver = webdriver.Chrome(options)
    return driver

In [31]:
# 2. Süreci optimize ederek, Travel ile Nonfiction kategorilerine ait tüm kitapların detaylarını alacak şekilde kodu düzenleyeceğiz

In [32]:
import time 
from selenium import webdriver
from selenium.webdriver.common.by import By

SLEEP_TIME = 0.25

def main():
    BASE_URL = "https://books.toscrape.com/"
    driver = initialize_driver()
    category_urls = get_travel_and_nonfiction_category_urls(driver, BASE_URL)
    data = []
    
    for cat_url in category_urls:
        book_urls = get_book_urls(driver , cat_url)
        for book_url in book_urls:
            book_data = get_book_detail(driver, book_url)
            book_data["cat_url"] = cat_url
            data.append(book_data)
    
    len(data)        
    
    
    import pandas as pd
    pd.set_option("display.max_columns" , None)
    pd.set_option("display.max_colwidth" , 40)
    pd.set_option("display.width" , 2000)
    df = pd.DataFrame(data)
    
    return df

    
df = main()

print(df.head())
print(df.shape)

TypeError: 'NoneType' object is not iterable

**Web Kazıma İşlemi**

In [36]:
import time 
from selenium import webdriver
from selenium.webdriver.common.by import By

SLEEP_TIME = 0.25

def initialize_driver():
    # Initilizes driver with maximized option
    options = webdriver.ChromeOptions()
    options.add_argument('--start-maximized')
    driver = webdriver.Chrome(options=options)
    return driver

def get_book_detail(driver, url):
    driver.get(url)
    time.sleep(SLEEP_TIME)
    content_div = driver.find_elements(By.XPATH , "//div[@class='content']")

    if not content_div:
        return None

    inner_html = content_div[0].get_attribute("innerHTML")
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(inner_html , "html.parser")
    name_elem = soup.find("h1")
    book_name = name_elem.text if name_elem else 'N/A'
    price_elem = soup.find("p" , attrs={"class":"price_color"})
    book_price = price_elem.text if price_elem else 'N/A'
    import re
    regex = re.compile('^star-rating')
    star_elem = soup.find("p" , attrs={"class":regex})
    book_star_count = star_elem["class"][-1] if star_elem else 'N/A'
    desc_elem = soup.find("div" , attrs={"id":"product_description"})
    book_desc = desc_elem.find_next_sibling().text if desc_elem else 'N/A'
    product_info = {}
    table_rows = soup.find("table").find_all("tr")
    for row in table_rows:
        key = row.find("th").text
        value = row.find("td").text
        product_info[key] = value    
    return {'book_name' : book_name , 'book_price' : book_price , 'book_star_count' : book_star_count , 'book_desc' : book_desc,
            **product_info}   

def get_book_urls(driver, url):
    MAX_PAGINATION = 3 #test amaçlı
    book_urls = []
    book_elements_xpath = "//div[@class='image_container']//a"

    for i in range(1 , MAX_PAGINATION + 1):
        update_url = url if i == 1 else url.replace("index.html" , f"page-{i}.html")
        driver.get(update_url)
        time.sleep(SLEEP_TIME)
        book_elements = driver.find_elements(By.XPATH, book_elements_xpath)
        
        if not book_elements:
            break
            
        temp_urls = [element.get_attribute("href") for element in book_elements]
        book_urls.extend(temp_urls)
    return book_urls

def get_travel_and_nonfiction_category_urls(driver , url):
    driver.get(url)
    time.sleep(SLEEP_TIME)
    category_elements_xpath ="//a[contains(text() , 'Travel') or contains(text() , 'Nonfiction')]"
    category_elements = driver.find_elements(By.XPATH , category_elements_xpath) # find.elements metodunu kullandık
    category_urls = [element.get_attribute("href") for element in category_elements]
    return category_urls

def main():
    BASE_URL = "https://books.toscrape.com/index.html"
    driver = initialize_driver()
    category_urls = get_travel_and_nonfiction_category_urls(driver, BASE_URL)
    data = []
    
    for cat_url in category_urls:
        book_urls = get_book_urls(driver , cat_url)
        for book_url in book_urls:
            book_data = get_book_detail(driver, book_url)
            if book_data:
                book_data["cat_url"] = cat_url
                data.append(book_data)
    
    import pandas as pd
    pd.set_option("display.max_columns" , None)
    pd.set_option("display.max_colwidth" , 40)
    pd.set_option("display.width" , 2000)
    df = pd.DataFrame(data)
    
    return df

df = main()

print(df.head())
print(df.shape)


                                 book_name book_price book_star_count                                book_desc               UPC Product Type Price (excl. tax) Price (incl. tax)    Tax             Availability Number of reviews                                  cat_url
0                  It's Only the Himalayas     £45.17             Two  “Wherever you go, whatever you do, j...  a22124811bfa8350        Books            £45.17            £45.17  £0.00  In stock (19 available)                 0  https://books.toscrape.com/catalogue...
1  Full Moon over Noah’s Ark: An Odysse...     £49.43            Four  Acclaimed travel writer Rick Antonso...  ce60436f52c5ee68        Books            £49.43            £49.43  £0.00  In stock (15 available)                 0  https://books.toscrape.com/catalogue...
2  See America: A Celebration of Our Na...     £48.87           Three  To coincide with the 2016 centennial...  f9705c362f070608        Books            £48.87            £48.87  £0.00  In stoc